# TOPS Stack Notebook

## Introduction
In this notebook, we will run all steps of processing Tops Stack. 
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

-----------------------------------

#### Note
Be sure to have ~/.netrc file include valid credentials for urs.earthdata.nasa.gov.

This notebook can process TOPS stacks for given spatial and temporal constraints.

**Input Parameters**

- **lat_min, lat_max, lon_min, lon_max**: min/max latitude and longitude of Region of Interest.


In [1]:
lat_min = 35.57793709766442
lat_max = 36.21619047354823
lon_min = -84.70058767311058
lon_max = -83.94773267597341

print(lat_min, lat_max, lon_min, lon_max)

35.57793709766442 36.21619047354823 -84.70058767311058 -83.94773267597341


In [2]:
# Parameters
lat_min = 35.57793709766442
lat_max = 36.21619047354823


## Initialization and set up

You might have to modify 00-init.sh

In [3]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh


Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83



topsStack requires certain input to run which are SLCs, Orbits, Dems etc.

## Fetch DEM

In [4]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/03-get-dems.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
dem.py -a stitch -b 34 38 -87 -83 -r -s 1 -c -f
API open (R): ./demLat_N34_N38_Lon_W087_W083.dem
API close:  ./demLat_N34_N38_Lon_W087_W083.dem
GDAL open (R): ./demLat_N34_N38_Lon_W087_W083.dem.vrt
API open (WR): demLat_N34_N38_Lon_W087_W083.dem.wgs84

    << Geoid Correction I2 SRTM>>

Jet Propulsion Laboratory - Radar Science and Engineering


Sampling Geoid at grid points -  Longitude Samples:    43 Latitude Lines:    43
Corner Geoid Heights (m) =  -30.37 -28.67 -32.50 -33.56

Correcting data to geoid height...

At line:      512
At line:     1024
At line:     1536
At line:     2048
At line:     2560
At line:     3072
At line:     3584
At line:     4096
At line:     4608
At line:     5120
At line:     5632
At line:     6144
At line:     6656
At line:     7168
At line:     7680
At line:     8192
At line:     8704
At line:     9216
At line:     9728
At line:    10240
At line:    10752
At line:    11264
At line:    11776
A

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   267  100   267    0     0   1628      0 --:--:-- --:--:-- --:--:--  1628
100   498  100   498    0     0   1134      0 --:--:-- --:--:-- --:--:--  1134
100   291    0   291    0     0    339      0 --:--:-- --:--:-- --:--:--  1182
100   267  100   267    0     0    211      0  0:00:01  0:00:01 --:--:--   211
100 8428k  100 8428k    0     0  3224k      0  0:00:02  0:00:02 --:--:-- 8584k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   267  100   267    0     0   2007      0 --:--:-- --:--:-- --:--:--  2022
100 8616k  100 8616k    0     0  6280k      0  0:00:01  0:00:01 --:--:-- 16.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   To

## Create run scripts and fetch orbits

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [5]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/04-make-run-scripts.sh
sh ${toolDir}/05-make-run-scripts.sh


Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
MASTER_DATE DOES NOT EXIST
stackSentinel.py -s zip/ -d demLat_N34_N38_Lon_W087_W083.dem.wgs84 -a AuxDir/ -o Orbits -b 34.285041 37.501509 -86.721161 -83.227722 -W slc -C geometry
Reference time:  2020-10-17 23:39:26
Satellite name:  S1A
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Number of SAFE files found: 4
*****************************************
20201017
orbit was not found in the /home/jovyan/run1-alt/Orbits
downloading precise or restituted orbits ...
fetchOrbit.py -i /home/jovyan/run1-alt/zip/S1A_IW_SLC__

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 's1qc.asf.alaska.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 's1qc.asf.alaska.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connecti

## Unpack TOPO reference

Unpacks the reference SLC files using ISCE readers, also unpackaging the antenna elevation pattern correction file if necessary. This run file also produces the reference geometry files that are consumed downstream.

Includes commands to unpack Sentinel-1 TOPS SLCs using ISCE readers. For older SLCs which need antenna elevation pattern correction, the file is extracted and written to disk. For newer version of SLCs which don’t need the elevation antenna pattern correction, only a gdal virtual “vrt” file (and isce xml file) is generated. The “.vrt” file points to the Sentinel SLC file and reads them whenever required during the processing. If a user wants to write the “.vrt” SLC file to disk, it can be done easily using gdal_translate (e.g. gdal_translate –of ENVI File.vrt File.slc). 
The “run_01_unpack_slc_topo_reference” also includes a command that refers to the config file of the stack reference, which includes configuration for running topo for the stack reference. Note that in the pair-wise processing strategy one should run topo (mapping from range-Doppler to geo coordinate) for all pairs. However, with stackSentinel, topo needs to be run only one time for the reference in the stack. 


In [6]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-01.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
sh run_files/run_01_unpack_topo_reference
Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/run1-alt/zip/S1A_IW_SLC__1SDV_20201005T233859_20201005T233926_034668_0409B0_2533.zip /home/jovyan/run1-alt/zip/S1A_IW_SLC__1SDV_20201005T233923_20201005T233950_034668_0409B0_9CD0.zip', '--swaths', '1 2 3', '--orbit', '/home/jovyan/run1-alt/orbits/20201005/S1A_OPER_AUX_POEORB_OPOD_20210317T225626_V20201005T225942_20201007T005942.EOF', '--outdir', '/home/jovyan/run1-alt/reference', '--auxdir', '/home/jovyan/run1-alt/AuxDir', '--bbox', '34.285041 37.501509 -86.721161 -83.227722', '--pol', 'vv']
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.n

/home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':


## Unpack secondary SLC

Unpack Secondary SLCs
In a manner similar to the SLCs in the step above, this step unpacks the secondary SLCs from each of the input SLC zip files.

In [7]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-0.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
1
cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/run1-alt/zip/S1A_IW_SLC__1SDV_20201017T233859_20201017T233926_034843_040FD4_E029.zip /home/jovyan/run1-alt/zip/S1A_IW_SLC__1SDV_20201017T233923_20201017T233950_034843_040FD4_DF43.zip', '--swaths', '1 2 3', '--orbit', '/home/jovyan/run1-alt/orbits/20201017/S1A_OPER_AUX_POEORB_OPOD_20210318T024542_V20201017T225942_20201019T005942.EOF', '--outdir', '/home/jovyan/run1-alt/secondarys/20201017', '--auxdir', '/home/jovyan/run1-alt/AuxDir', '--bbox', '34.285041 37.501509 -86.721161 -83.227722', '--pol', 'vv']
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the w


Computers / CPU cores / Max jobs to run
1:local / 36 / 1

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 1 AVG: 0.00s  local:1/0/100%/0.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 0 AVG: 0.00s  local:0/1/100%/62.0s 


## SLC noise calibration

This step runs radiometric and thermal noise calibration using vh-pol data against the SLCs.

In [8]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-5.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of


Computers / CPU cores / Max jobs to run
1:local / 36 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 2 AVG: 0.00s  local:2/0/100%/0.0s usage: read_calibration_slc.py [-h] [-i FXML] [-is FXMLSCENE] [-zip FIN [FIN ...]] [-od ODIR] [-o] [-ext EXT [EXT ...]]
                               [-n SWATH_NUM] [-p POL] [-t TYPE] [-oc] [-ck]
read_calibration_slc.py: error: argument -zip/--zipfile: expected at least one argument
ETA: 465s Left: 1 AVG: 506.00s  local:1/1/100%/506.0s /home/jovyan/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar/topsStack/read_calibration_slc.py:341: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  for child in caliRoot.getchildren():
/home/jovyan/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar/topsStack/read_calibration_slc.py:403: DeprecationWarning: This method will be removed in future versions.  Use 'list(el

## Discard subswaths not to be used

In [9]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-9.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
discard subswaths not to be used
IW dirs under reference dir: {'IW2', 'IW3', 'IW1'}
IW dirs under secondary dirs: {'20201017': {'IW2', 'IW3', 'IW1'}}
common IW dirs: {'IW2', 'IW3', 'IW1'}
dirs to delete: []
0


## Average baseline

Computes average baseline for the stack. These baselines are not used for processing anywhere. They are only an approximation and can be used for plotting purposes.

In [10]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-03.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/run1-alt/reference/', '--secondary', '/home/jovyan/run1-alt/secondarys/20201017', '--baseline_file', '/home/jovyan/run1-alt/baselines/20201005_20201017/20201005_20201017.txt']
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
minSecondary,maxSecondary 1 18
minReference,maxReference 1 18
minBurst, maxBurst:  1 18
Bpr


Computers / CPU cores / Max jobs to run
1:local / 36 / 1

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 1 AVG: 0.00s  local:1/0/100%/0.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 0 AVG: 0.00s  local:0/1/100%/11.0s 


## fullBurst geo2rdr

This step estimates geometrical offsets between secondary burst overlaps and the stack reference burst overlaps. The secondaries are then resampled to the stack reference burst overlaps.

In [11]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-04.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%
API open (WR): /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/range_01.off
API open (WR): /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/azimuth_01.off
GDAL open (R): /home/jovyan/run1-alt/geom_reference/IW1/hgt_01.rdr.vrt
GDAL open (R): /home/jovyan/run1-alt/geom_reference/IW1/lat_01.rdr.vrt
GDAL open (R): /home/jovyan/run1-alt/geom_reference/IW1/lon_01.rdr.vrt
API close:  /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/range_01.off
API open (R): /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/range_01.off
API close:  /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/range_01.off
API close:  /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/azimuth_01.off
API open (R): /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/azimuth_01.off
API close:  /home/jovyan/run1-alt/coreg_secondarys/20201017/IW1/azimuth_01.off
GDAL close: /home


Computers / CPU cores / Max jobs to run
1:local / 36 / 1

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 1 AVG: 0.00s  local:1/0/100%/0.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 0 AVG: 0.00s  local:0/1/100%/1069.0s 


## fullBurst resample

Using orbit and DEM data, this run file computes geometrical offsets among all secondary SLCs and the stack reference. These offsets, with the misregistration time series are used for precise coregistration of each burst SLC.


In [12]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-05.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/run1-alt/secondarys/20201017', '--reference', '/home/jovyan/run1-alt/reference', '--coregdir', '/home/jovyan/run1-alt/coreg_secondarys/20201017', '--azimuth_misreg', '/home/jovyan/run1-alt/misreg/azimuth/dates/20201017.txt', '--range_misreg', '/home/jovyan/run1-alt/misreg/range/dates/20201017.txt']
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
h


Computers / CPU cores / Max jobs to run
1:local / 36 / 1

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 1 AVG: 0.00s  local:1/0/100%/0.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 0 AVG: 0.00s  local:0/1/100%/464.0s 


## Extract stack valid region

In [13]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-06.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
sh run_files/run_06_extract_stack_valid_region
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
creating  /home/jovyan/run1-alt/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  1
/home/jovyan/run1-alt/coreg_secondarys/20201017
minSecondary,maxSecondary 1 18
minReference,maxReference 1 18
minBurst, maxBurst:  1 18
writing  /home/jovyan/run1-alt/stack/IW1.xml
******************
swath:  2
/h

## Merge reference secondary SLC

This step merges all bursts for the reference and coregistered SLCs. Geometry files are also merged. Using orbit and DEM, geometrical offsets among all secondary SLCs and the stack reference is computed. The geometrical offsets, together with the misregistration time-series (from previous step) are used for precise coregistration of each burst SLC. 

In [14]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-07.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
cat run_files/run_07_merge_reference_secondary_slc | parallel -j2 --eta --load 50%
Input file size is 68843, 25190
0...10...20...30...40...50...60...70...80...90...100 - done.
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/run1-alt/stack', '--inp_reference', '/home/jovyan/run1-alt/reference', '--dirname', '/home/jovyan/run1-alt/geom_reference', '--name_pattern', 'lat*rdr', '--outfile', '/home/jovyan/run1-alt/merged/geom_reference/lat.rdr', '--method', 'top', '--multilook', '--range_looks', '9', '--azimuth_looks', '3', '--multilook_tool', 'gdal', '--no_data_value', '0']
Using default ISCE Path: /opt/isce2/isce
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm


Computers / CPU cores / Max jobs to run
1:local / 36 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 8 AVG: 0.00s  local:2/0/100%/0.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 802s Left: 7 AVG: 125.00s  local:2/1/100%/248.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 932s Left: 6 AVG: 162.50s  local:2/2/100%/224.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 677s Left: 5 AVG: 146.00s  local:2/3/100%/187.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 489s Left: 4 AVG: 123.25s  local:2/4/100%/154.0s /home/jovyan/isce2/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a lit

## Create dataset

The final step ‘packages’ up the results into a single sub-directory named as coregistered_slcs-<start_date>-<end_date> and generates json files describing the dataset (e.g. the polygon  coordinates of the bounding box) and associated metadata (e.g. included SLCs).

More information about metadata files can be found at : https://hysds-core.atlassian.net/wiki/spaces/HYS/pages/389349377/Product+Metadata+Files

In [15]:
%%bash

toolDir=~/tops-stack-processor-hamsar-develop-conda-isce2/topsstack-hamsar

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

# fake env var PROCESSING_START for now
export PROCESSING_START=$(date +%FT%T)

sh ${toolDir}/07-make-datasets.sh

Coords:
34.285041 37.501509 -86.721161 -83.227722 34 38 -87 -83
{
  "version": "v1.0",
  "starttime": "2020-10-05T23:38:59.095898",
  "endtime": "2020-10-17T23:39:50.944080",
  "location": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -86.721161,
          37.501509
        ],
        [
          -83.227722,
          37.501509
        ],
        [
          -83.227722,
          34.285041
        ],
        [
          -86.721161,
          34.285041
        ],
        [
          -86.721161,
          37.501509
        ]
      ]
    ]
  },
  "label": "coregistered_slcs-20201005233859-20201017233950"
}
{
  "processing_start": "2021-06-15T07:01:47",
  "processing_stop": "2021-06-15T07:01:47",
  "version": "v1.0",
  "direction": "asc",
  "orbit_number": 34668,
  "track_number": 121,
  "sensor": "SAR-C Sentinel1",
  "platform": "Sentinel-1A",
  "orbit_cycles": [
    212,
    213
  ],
  "bbox": [
    [
      36.109687501080124,
      -83.55734481047764
    ],
 